In [13]:
from Bio import motifs
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd


In [19]:
def write_sequence_fasta(fasta_file, tss_regions_sequences, tss_names):
    """
    Write fasta format file with the extendede sequences from the TSS peaks
    :param fasta_file: name of the output file
    :param tss_regions_sequences: the extended TSS region sequences
    :param tss_names: the names of the TSS peaks (or coordinates)
    :return: null
    """
    nf = open(fasta_file, 'w')
    for i in range(len(tss_regions_sequences)):
        tmp_name = tss_names[i]
        tmp_reg = str(tss_regions_sequences[i])
        nf.write(">" + tmp_name + "\n")
        nf.write(tmp_reg + "\n")
    nf.close()

def extract_extended_regions(tss_coords, extension_range, genome_dict):
    """
    Extract sequence of extended regions around TSS peaks from CAGE
    :param tss_coords: the coordinates in format chr_start_end_strand
    :param extension_range: the range of extension in bp
    :param genome_dict: the dictionary containing the genome sequences
    :return: Series of extended regions as Seq objects
    """
    tss_regions = []
    for tssc in tss_coords:
        tss = tssc.split("_")
        chr = tss[0]
        start = int(tss[1]) - extension_range - 1
        end = int(tss[2]) + extension_range - 1
        strand = tss[3]
        rec = genome_dict[chr]
        tss_reg = rec.seq[start:end]
        tss_regions.append(tss_reg)

    return pd.Series(tss_regions)


In [15]:
    ## Some currently hard-coded variables
    genome_file = "/home/kevin/resources/genomes/GRCh38_v27_gencode/GRCh38.primary_assembly.genome.fa"
    peak_file = "/home/kevin/rimod/CAGE/cage_analysis/CAGE_deseq_analysis_2018-03-07_11.16.24/deseq_result_grn.ndc_2018-03-07_11.16.24.txt"
    extension_range = 1000
    output_filename = "/home/kevin/rimod/CAGE/cage_analysis/sequence_sig_peaks_up.fasta"
    jasp_motifs = "/home/kevin/resources/TF_interactions/JASPAR/JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt"

    # Load the genome
    print("Loading genome from " + genome_file)
    genome_dict = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))
    print("Genome loaded")

    # Read peak file peak file
    peak_df = pd.read_table(peak_file)
    peak_df = peak_df.rename(columns={peak_df.columns.tolist()[0]: 'tss_coordinates'})

    # Only consider significant, up-regulated peaks
    peak_df = peak_df.loc[peak_df['padj'] <= 0.05]
    peak_df = peak_df.loc[peak_df['log2FoldChange'] >= 1]
    peak_df.index = pd.RangeIndex(peak_df.shape[0])
    tss_coords = peak_df['tss_coordinates']

Loading genome from /home/kevin/resources/genomes/GRCh38_v27_gencode/GRCh38.primary_assembly.genome.fa


Genome loaded


In [86]:
    # Extract sequence of extended regions
    print("Extracting regions ...")
    tss_regions = extract_extended_regions(tss_coords, extension_range, genome_dict)
    tss_coords = pd.Series(tss_coords)
    print("Regions extracted.")

Extracting regions ...
Regions extracted.


Save sequence of extended regions as fasta

In [20]:
    # Save region sequences as fasta file
    print("Writing extended sequences to fasta ...")
    write_sequence_fasta(output_filename, tss_regions, tss_coords)

Writing extended sequences to fasta ...


#Load motifs

In [115]:
fh = open(jasp_motifs)
tf_motifs = motifs.parse(fh, 'jaspar')
# Small numbers for debugging
tf_motifs = tf_motifs[0:5]
tss_regions = tss_regions[0:100]

In [126]:
def get_strand(x):
    return x.split("_")[3]
strand = tss_coords.apply(get_strand)
pos_idx = strand[strand == '+']
len(pos_idx)

1744

For every motif, look for instances in the extended sequences and save the result

In [118]:
def count_gen(iter):
    count = 0
    for elem in iter:
        if elem[1] > 0:
            count +=1      
    return count

# First transform sequence alphabets
print("Setting correct sequence alphabets ... ")
m = tf_motifs[0]
for seq in tss_regions:
    seq.alphabet = m.alphabet

print("Calculating odds scores ...")
tf_dict = {}
background = {'A':0.3, 'C':0.2, 'G':0.2, 'T':0.3}
pseudocounts = {'A':0.6, 'C':0.4, 'G':0.4, 'T':0.6}
log_odds_score = 6.0
for m in tf_motifs:
    score_acc = 0
    pwm = m.counts.normalize(pseudocounts = pseudocounts)
    pssm = pwm.log_odds(background)
    threshold = pssm.distribution(background=background, precision=10**4).threshold_fpr(0.001)
    print(str(threshold))
    print(m.name)
    nf = open(m.name + "_hits.txt", 'w')
    nf.write("TF\tTSS_Peak\tHits\n")
    for idx,seq in enumerate(tss_regions):
        hits = count_gen(pssm.search(seq, threshold))
        score_acc += hits
        #nf.write(m.name + "\t" + tss_coords[idx] + "\t" + str(hits) + "\n")
    tf_dict[m.name] = score_acc
    nf.close()

Setting correct sequence alphabets ... 
Calculating odds scores ...


6.648687713554327
Arnt


6.514705908507054
Ahr::Arnt


6.618161102349596
Ddit3::Cebpa


6.782247574699987
NFIL3


KeyboardInterrupt: 

In [108]:
df = pd.Series(tf_dict).to_frame()
df.columns = ['Score']

In [109]:
df

,Score
Ahr::Arnt,192
Arnt,310
Ddit3::Cebpa,116
FOXD1,72
FOXF2,47
Foxd3,140
Foxq1,56
Gfi1,103
Mecom,22
NFIL3,25
